### Inicializando ChromeDriver

In [1]:

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# → Inicializa o ChromeDriver automaticamente
servico = Service(ChromeDriverManager().install())

# → Cria o navegador
navegador = webdriver.Chrome(service=servico)


### Capturando Links

In [4]:


navegador.get(r'https://www.bcb.gov.br/publicacoes/atascopom/cronologicos')

lista = navegador.find_elements(By.CLASS_NAME, 'mb-3')

links = []
for elemento in lista:
  
    tag_a = elemento.find_element(By.TAG_NAME, 'a')
    link = tag_a.get_attribute('href')
    links.append(link)
    


#### Conferindo Links

In [ ]:
print(len(links))
for i in links:
    print(i)



### Colentando Links por PDF's das páginas em que não há texto, são as atas 200-231
####  Se faz necessário pois a ata está somente disponível via PDF
 OBS: Caso o Copom Adicionar mais uma ata os índices devem ser alterados

In [5]:

links_pdf = []
links_partido = links[44:76]

for link_p in links_partido:

    
    navegador.get(link_p)
   
    wait = WebDriverWait(navegador, 20)

    
    ata = wait.until(
        EC.presence_of_element_located((By.TAG_NAME, 'download'))
    )


    tag_a = ata.find_element(By.TAG_NAME, 'a')
    
    link_pdf = tag_a.get_attribute('href')
    links_pdf.append(link_pdf)

    
d = {
    'link_site': links_partido,
    'link_pdf':links_pdf
}

    

#### Conferindo links

In [6]:
print(len(links_partido))
for i in links_partido:
    print(i)

32
https://www.bcb.gov.br/publicacoes/atascopom/17062020
https://www.bcb.gov.br/publicacoes/atascopom/06052020
https://www.bcb.gov.br/publicacoes/atascopom/18032020
https://www.bcb.gov.br/publicacoes/atascopom/05022020
https://www.bcb.gov.br/publicacoes/atascopom/11122019
https://www.bcb.gov.br/publicacoes/atascopom/30102019
https://www.bcb.gov.br/publicacoes/atascopom/18092019
https://www.bcb.gov.br/publicacoes/atascopom/31072019
https://www.bcb.gov.br/publicacoes/atascopom/19062019
https://www.bcb.gov.br/publicacoes/atascopom/08052019
https://www.bcb.gov.br/publicacoes/atascopom/20032019
https://www.bcb.gov.br/publicacoes/atascopom/06022019
https://www.bcb.gov.br/publicacoes/atascopom/12122018
https://www.bcb.gov.br/publicacoes/atascopom/31102018
https://www.bcb.gov.br/publicacoes/atascopom/19092018
https://www.bcb.gov.br/publicacoes/atascopom/01082018
https://www.bcb.gov.br/publicacoes/atascopom/20062018
https://www.bcb.gov.br/publicacoes/atascopom/16052018
https://www.bcb.gov.br/pu

#### Fazendo o donwload dos PDF's e armazenando eles em uma pasta

In [21]:

from pathlib import Path
import requests
filename = Path('PDFS das ATAS 231-200 (teste)')

num = list(range(231,199,-1))


for link_pdf in d['link_pdf']:
    response = requests.get(link_pdf)
    i = num[0]
    with open(filename / f'Ata_Copom_{i}(teste).pdf', 'wb') as f:
        f.write(response.content)
    num.remove(num[0])


### Extraindo Texto dos PDFs

In [ ]:
# importing required modules
from pypdf import PdfReader
from pathlib import Path
import json
# creating a pdf reader object
arquivos = Path('PDFS das ATAS 231-200')

num = list(range(200,233))

i = len(d['link_site'])

with open('atas 231-200.jsonl', 'w', encoding='utf-8') as f:
    for pdf in arquivos.iterdir():
        i = i-1
        reader = PdfReader(pdf)

        pages = reader.pages
    
        pages = pages[1:len(pages)]
       
        agrupando_text = ''
        
        for pagina in pages:
            text = pagina.extract_text()
            
            agrupando_text += text + '\n'
        
        n = num[0]
        
        d2 = {
            "N°": n,
            "url": d['link_site'][i],
            "tamanho": len(agrupando_text),
            "conteudo": agrupando_text
        }


        f.write(json.dumps(d2, ensure_ascii=False)+ "\n")
        num.remove(num[0])

        
        

### Coletando Atas por texto do site
Foram coletadas as atas da última publicada até a ata 232

In [ ]:
# Páginas entre 274-232
import json
atas = []

with open('atas_275-232.jsonl', 'w', encoding='utf-8') as f:
    for link in links[0:44]:
        agrupando_paragrafos = ''

        
        navegador.get(link)
        
        wait = WebDriverWait(navegador, 20)

        
        wait.until(
            EC.presence_of_element_located((By.ID, 'ataconteudo'))
        )

        ata = navegador.find_element(By.ID, 'ataconteudo')
        texto = ata.text
        
        
        i = links.index(link)
        d = {
            "N°": 275 - i,
            "url": link,
            "tamanho": len(texto),
            "conteudo": texto
        }
        
        
        f.write(json.dumps(d, ensure_ascii=False) + "\n")
    
        print(f'ata {len(links)-i} baixada!')
        
    

In [8]:
numeração = list(range(275,231,-1))
print(numeração)

[275, 274, 273, 272, 271, 270, 269, 268, 267, 266, 265, 264, 263, 262, 261, 260, 259, 258, 257, 256, 255, 254, 253, 252, 251, 250, 249, 248, 247, 246, 245, 244, 243, 242, 241, 240, 239, 238, 237, 236, 235, 234, 233, 232]


In [ ]:

numeração = list(range(275,231,-1))

n = {'N°':numeração}

d = {
    'N°':numeração,
    'ATA CONTEUDO':atas
}

df = pd.DataFrame(d)
print(df)

In [45]:
df.to_json(
    "Atas_274-232.jsonl",
    orient="records",
    lines=True,
    force_ascii=False
)

### Coletando Atas 199 até 21

In [ ]:
for i,link in enumerate(links[75:]):
    print(i,link)

In [ ]:
import json
atas = []


with open('atas_199-21.jsonl', 'w', encoding='utf-8') as f:
    for link in links[76:]:
        agrupando_paragrafos = ''

        
        navegador.get(link)
        
        wait = WebDriverWait(navegador, 20)

        
        wait.until(
            EC.presence_of_element_located((By.ID, 'publicacao'))
        )

        ata = navegador.find_elements(By.ID, 'publicacao')
        
        
        for paragrafo in ata:
            
            texto = paragrafo.text 
            agrupando_paragrafos +=  texto 
        
        i = links.index(link)
        d = {
            "N°": 275 - i,
            "url": link,
            "tamanho": len(agrupando_paragrafos),
            "conteudo": agrupando_paragrafos
        }
        
        f.write(json.dumps(d, ensure_ascii=False) + "\n")
        
        
        print(f'ata {275-i} baixada!')
    
        

In [100]:
import pandas as pd

numeração = list(range(199,97-1,-1))

n = {'N°':numeração}

d = {
    'N°':numeração,
    'ATA CONTEUDO':atas
}


df = pd.DataFrame(d)


In [101]:



df.to_json(
    "Atas_199-98.jsonl",
    orient="records",
    lines=True,
    force_ascii=False
)

### Merge dos dados

In [ ]:
from pathlib import Path

arquivos = [
    Path('atas_275-232.jsonl'),
    Path('atas 231-200.jsonl'),
    Path('atas_199-21.jsonl'),
]

saida = Path("Atas_Copom_275-21.jsonl")

with open(saida, "w", encoding="utf-8") as fout:
    for arq in arquivos:
        with open(arq, "r", encoding="utf-8") as fin:
            for linha in fin:
                fout.write(linha)
